# Beam Simulations
Getting started with notes from James Lamb.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import fft

## Start with 1.35 GHz, ideal beam.

### Note on various sizes
EXTENT_M is the extent of the illumination pattern grid in meters. This controls the E beam spatial resolution.


In [2]:
coeffs = np.array([0.41014, 0.09540, -0.27752, 0.17076, -0.05474, 0.00988, -0.00095])

In [1]:
EXTENT_M = 250
GRID_SIZE = 1000
LAMBD = 3e8/1.35e9
DIAM_M = 5

In [3]:
grid_cell = EXTENT_M / (GRID_SIZE//2)

In [5]:
ea_grid_m = np.meshgrid(np.linspace(-DIAM_M, DIAM_M, num=GRID_SIZE + 1), np.linspace(-DIAM_M, DIAM_M, num=GRID_SIZE + 1))
ea_grid = np.sum(np.power(np.sqrt(ea_grid_m[0][:-1, :-1]**2 + ea_grid_m[1][:-1, :-1]**2), np.arange(1, len(coeffs) + 1)[:,np.newaxis, np.newaxis]), axis=0)
ea_grid[np.sqrt(ea_grid_m[0][:-1,:-1]**2 + ea_grid_m[1][:-1, :-1]**2) > 5] = 0

In [6]:
%matplotlib widget
fig, ax = plt.subplots()
ax.set_aspect(1)
p = ax.pcolor(ea_grid_m[0], ea_grid_m[1], np.log10(ea_grid))
ax.set_xlabel('X (m)')
ax.set_ylabel('y (m)')
ax.set_title('Antenna Illumination electric field (Ea)')
fig.colorbar(p)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-6-bf2664f5b7b3>:4: RuntimeWarning: divide by zero encountered in log10
  p = ax.pcolor(ea_grid_m[0], ea_grid_m[1], np.log10(ea_grid))


Zero pad and then FFT

In [ ]:
def to_beam(illum, extent_m):
    pad_size = (GRID_SIZE - illum.shape[0]) // 2
    illum_padded = np.pad(illum, pad_size)
    assert GRID_SIZE == illum_padde.shape[0], ''
    beam = fft.fft2(illum_padded)
    f = fft.fftfreq(GRID_SIZE, grid_cell/(2 * np.pi * LAMBD))
    freqs = fft.fftshift(np.meshgrid(f, f))
    return freqs, beam

In [10]:
freqs, e_beam = to_beam(ea_grid,  

In [16]:
ft_grid = fft.fftshift(np.meshgrid(freqs, freqs))

In [13]:
mfreq = freqs.max() * 57.2958

In [14]:
#d1 = np.argsort(freqs)
#d2 = np.argsort(freqs)

In [32]:
%matplotlib widget
fig, ax = plt.subplots()
plt.imshow(np.abs(e_beam), extent=[-mfreq, mfreq, -mfreq, mfreq])
ax.set_xlabel('deg')
ax.set_ylabel('deg')
ax.set_title('Far-field electric field (E)')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
%matplotlib widget
# the arctan gives the angle and avoids the 0-pi confusion.
plt.imshow(np.arctan((e_beam.imag/e_beam.real)), extent=[-mfreq, mfreq, -mfreq, mfreq])
plt.colorbar()
ax.set_xlabel('deg')
ax.set_ylabel('deg')
ax.set_title('Far-field electric field (E)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Far-field electric field (E)')

## All the extra phase stuff
For everything else, I should be able to imprint an extra phase $\theta(x,y)=\Delta p(x,y)$ on the illumination pattern, and then FT.

#### Surface error

# Fitting the power beam

In [16]:
from scipy import optimize
def gaussian(height, center_x, center_y, width_x, width_y):
    """Returns a gaussian function with the given parameters"""
    width_x = float(width_x)
    width_y = float(width_y)
    return lambda x,y: height*np.exp(
                -(((center_x-x)/width_x)**2+((center_y-y)/width_y)**2)/2)

def moments(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution by calculating its
    moments """
    total = data.sum()
    X, Y = np.indices(data.shape)
    x = (X*data).sum()/total
    y = (Y*data).sum()/total
    col = data[:, int(y)]
    width_x = np.sqrt(np.abs((np.arange(col.size)-x)**2*col).sum()/col.sum())
    row = data[int(x), :]
    width_y = np.sqrt(np.abs((np.arange(row.size)-y)**2*row).sum()/row.sum())
    height = data.max()
    return height, x, y, width_x, width_y

def fitgaussian(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution found by a fit"""
    params = moments(data)
    errorfunction = lambda p: np.ravel(gaussian(*p)(*np.indices(data.shape)) -
                                 data)
    p, success = optimize.leastsq(errorfunction, params)
    return p

In [26]:
data = amp[d1][:,d2][450:550, 450:550] ** 2

In [27]:
params = fitgaussian(data)

In [28]:
params

array([3.76035193e+11, 4.99989148e+01, 4.99989148e+01, 1.37405372e+01,
       1.37405372e+01])

In [29]:
amp.shape

(1000, 1000)

In [34]:
%matplotlib widget
fit = gaussian(*params)
plt.imshow(data)
plt.contour(fit(*np.indices(data.shape)), cmap=plt.cm.copper)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
params[3] * freqs[1] * 57.2958

2.198486737514539

In [35]:
data.shape

(100, 100)

In [41]:
%matplotlib widget
plt.plot(np.linspace(-50, 50, 100) * freqs[1] * 57.2958, data[data.shape[0]//2,:]/data[data.shape[0]//2,:].max())
plt.grid()
plt.xlabel('deg')
plt.ylabel('Normalized Power')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Normalized Power')